<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/CNN-inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import Conv1D, Dense, AveragePooling1D
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
scaler = StandardScaler()
encoder = LabelEncoder()

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/train-intervehicle.csv',chunksize=500000)
test = pd.read_csv('/content/drive/MyDrive/test-intervehicle.csv')

In [ ]:
model = Sequential()
model.add(Conv1D(activation="sigmoid", input_shape=(len(test.columns)-1,1), kernel_size=5, filters=6))
model.add(AveragePooling1D(pool_size=2))
model.add(Conv1D(activation="sigmoid", kernel_size=5, filters=6))
model.add(AveragePooling1D(pool_size=2))
model.add(Conv1D(activation="sigmoid", kernel_size=5, filters=12))
model.add(AveragePooling1D(pool_size=2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 66, 6)             36        
                                                                 
 average_pooling1d (AverageP  (None, 33, 6)            0         
 ooling1D)                                                       
                                                                 
 conv1d_1 (Conv1D)           (None, 29, 6)             186       
                                                                 
 average_pooling1d_1 (Averag  (None, 14, 6)            0         
 ePooling1D)                                                     
                                                                 
 conv1d_2 (Conv1D)           (None, 10, 12)            372       
                                                                 
 average_pooling1d_2 (Averag  (None, 5, 12)            0

In [ ]:
for chunk in train:
  train_y = chunk.pop('Label')
  train_x = chunk
  train_x = scaler.fit_transform(train_x)
  train_y = encoder.fit_transform(train_y)
  model.fit(train_x, train_y, batch_size=32)

6161/6161 [==============================] - 19s 3ms/step - loss: 7.8231 - accuracy: 0.4928


In [ ]:
test_y = test.pop('Label')
test_x = test

In [ ]:
test_x = scaler.transform(test_x)
test_y = encoder.transform(test_y)

In [ ]:
pred = model.predict(test_x, verbose=1)
pred_y = np.argmax(pred, axis=1)

79666/79666 [==============================] - 225s 3ms/step - loss: 7.8270 - accuracy: 0.4926
Score 7.8269524574279785 Accuracy 0.4925849735736847


In [ ]:
confusion_matrix(test_y, pred_y)

In [ ]:
report = classification_report(test_y, pred_y)